In [20]:
import pandas as pd
import copy
import random

In [51]:
# constsant
ingr_percent = 1
good_rat = 5.0
bad_rat = 0.0
ingr_count = 5
calories = [2000, 2400, 2800] # len(calories) = nutr_count
mult = 2/3
nutr_count = 3
food_count = 20

In [3]:
ingr_data = pd.read_csv('../data/ingr_data.csv')
ingr_data.sort_values(by = 'count', ascending = False, inplace = True)
ingr_data.head()

,iid,name,count
6270,6270,salt,72848
840,840,butter,48039
2499,2499,egg,43350
5010,5010,onion,42631
6906,6906,sugar,37464


In [4]:
ingr_candidate = ingr_data.head(int(len(ingr_data)*(ingr_percent/100)))
ingr_candidate

,iid,name,count
6270,6270,salt,72848
840,840,butter,48039
2499,2499,egg,43350
5010,5010,onion,42631
6906,6906,sugar,37464
...,...,...,...
8021,8021,zucchini,3948
2509,2509,egg white,3925
5825,5825,raisin,3894
3668,3668,heavy cream,3829


In [61]:
ingr_list = ingr_candidate['iid'].tolist()

In [62]:
ingr_list[0:10]

[6270, 840, 2499, 5010, 6906, 5006, 3203, 7655, 4717, 2683]

In [22]:
recipe_data = pd.read_csv('../data/recipe_data.csv')
recipe_data.set_index('fid', inplace = True)
recipe_data.head()

,name,full_id,nutrition,ingredient_ids
fid,,,,
0,white bean green chile pepper soup,40893,"[204.8, 5.0, 9.0, 26.0, 24.0, 2.0, 10.0]","[3384, 7979, 2127, 3502, 3217, 1257, 2778, 500..."
1,devilicious cookie cake delights,44394,"[132.3, 11.0, 39.0, 5.0, 4.0, 11.0, 5.0]","[912, 7557, 2499, 5382]"
2,baked potato toppings,85009,"[2786.2, 342.0, 134.0, 290.0, 161.0, 301.0, 42.0]","[4623, 6265, 1168, 6016, 3597, 3440, 7213, 169..."
3,kfc honey bbq strips,134728,"[316.0, 4.0, 40.0, 37.0, 78.0, 4.0, 10.0]","[1304, 2683, 3217, 6270, 3532, 869, 7557, 3698..."
4,lamb stew with tomatoes chickpeas and spices,200236,"[606.5, 65.0, 12.0, 34.0, 65.0, 83.0, 7.0]","[4130, 6270, 3486, 7557, 5010, 3203, 2683, 125..."


In [9]:
recipe_data['ingredient_ids'] = recipe_data['ingredient_ids'].str.replace(" ", "")
recipe_data['ingredient_ids'] = recipe_data['ingredient_ids'].apply(lambda x: x[1:-1].split(','))

In [10]:
recipe_data['nutrition'] = recipe_data['nutrition'].str.replace(" ", "")
recipe_data['nutrition'] = recipe_data['nutrition'].apply(lambda x: x[1:-1].split(','))

In [11]:
recipe_data.head()

,name,full_id,nutrition,ingredient_ids
fid,,,,
0,white bean green chile pepper soup,40893,"[204.8, 5.0, 9.0, 26.0, 24.0, 2.0, 10.0]","[3384, 7979, 2127, 3502, 3217, 1257, 2778, 500..."
1,devilicious cookie cake delights,44394,"[132.3, 11.0, 39.0, 5.0, 4.0, 11.0, 5.0]","[912, 7557, 2499, 5382]"
2,baked potato toppings,85009,"[2786.2, 342.0, 134.0, 290.0, 161.0, 301.0, 42.0]","[4623, 6265, 1168, 6016, 3597, 3440, 7213, 169..."
3,kfc honey bbq strips,134728,"[316.0, 4.0, 40.0, 37.0, 78.0, 4.0, 10.0]","[1304, 2683, 3217, 6270, 3532, 869, 7557, 3698..."
4,lamb stew with tomatoes chickpeas and spices,200236,"[606.5, 65.0, 12.0, 34.0, 65.0, 83.0, 7.0]","[4130, 6270, 3486, 7557, 5010, 3203, 2683, 125..."


In [23]:
PP_users = pd.read_csv('../data/food/PP_users.csv')
user_data = PP_users.set_index('u').loc[:, ['items', 'ratings']]
user_data.head()

,items,ratings
u,,
0,"[1118, 27680, 32541, 137353, 16428, 28815, 658...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 4.0, ..."
1,"[122140, 77036, 156817, 76957, 68818, 155600, ...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ..."
2,"[168054, 87218, 35731, 1, 20475, 9039, 124834,...","[3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 5.0, ..."
3,"[163193, 156352, 102888, 19914, 169438, 55772,...","[5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 5.0, 5.0, ..."
4,"[72857, 38652, 160427, 55772, 119999, 141777, ...","[5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 4.0, 5.0, ..."


In [24]:
user_data['items'] = user_data['items'].str.replace(" ", "")
user_data['items'] = user_data['items'].apply(lambda x: x[1:-1].split(','))

In [25]:
user_data['ratings'] = user_data['ratings'].str.replace(" ", "")
user_data['ratings'] = user_data['ratings'].apply(lambda x: x[1:-1].split(','))

In [26]:
user_data.head()

,items,ratings
u,,
0,"[1118, 27680, 32541, 137353, 16428, 28815, 658...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 4.0, ..."
1,"[122140, 77036, 156817, 76957, 68818, 155600, ...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ..."
2,"[168054, 87218, 35731, 1, 20475, 9039, 124834,...","[3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 5.0, ..."
3,"[163193, 156352, 102888, 19914, 169438, 55772,...","[5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 5.0, 5.0, ..."
4,"[72857, 38652, 160427, 55772, 119999, 141777, ...","[5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 4.0, 5.0, ..."


In [23]:
# ingredient related constraints need to be consistent
# ex) included ingredient shouldn't have bad rating
# ex) excluded ingredient shouldn't have good rating

In [130]:
ingr_column = [copy.deepcopy(ingr_list) for x in range(len(user_data))]
len(ingr_column)

25076

In [131]:
user_data = user_data.assign(include = ingr_column)
user_data = user_data.assign(exclude = copy.deepcopy(ingr_column))
user_data.head()

,items,ratings,include,exclude
u,,,,
0,"[1118, 27680, 32541, 137353, 16428, 28815, 658...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 4.0, ...","[6270, 840, 2499, 5010, 6906, 5006, 3203, 7655...","[6270, 840, 2499, 5010, 6906, 5006, 3203, 7655..."
1,"[122140, 77036, 156817, 76957, 68818, 155600, ...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[6270, 840, 2499, 5010, 6906, 5006, 3203, 7655...","[6270, 840, 2499, 5010, 6906, 5006, 3203, 7655..."
2,"[168054, 87218, 35731, 1, 20475, 9039, 124834,...","[3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 5.0, ...","[6270, 840, 2499, 5010, 6906, 5006, 3203, 7655...","[6270, 840, 2499, 5010, 6906, 5006, 3203, 7655..."
3,"[163193, 156352, 102888, 19914, 169438, 55772,...","[5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 5.0, 5.0, ...","[6270, 840, 2499, 5010, 6906, 5006, 3203, 7655...","[6270, 840, 2499, 5010, 6906, 5006, 3203, 7655..."
4,"[72857, 38652, 160427, 55772, 119999, 141777, ...","[5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 4.0, 5.0, ...","[6270, 840, 2499, 5010, 6906, 5006, 3203, 7655...","[6270, 840, 2499, 5010, 6906, 5006, 3203, 7655..."


In [132]:
for index, row in user_data.iterrows():
    for x in range (0, len(row['items'])):
        item_id = int(row['items'][x])
        if float(row['ratings'][x]) == good_rat : # remove ingredients included in the item frome exclude list
            ingrs = recipe_data.loc[item_id].ingredient_ids
            for ingr in ingrs:
                if int(ingr) in row['exclude']:
                    row['exclude'].remove(int(ingr))
        if float(row['ratings'][x]) == bad_rat :
            ingrs = recipe_data.loc[item_id].ingredient_ids
            for ingr in ingrs:
                if int(ingr) in row['include']:
                    row['include'].remove(int(ingr))

In [133]:
user_data.head()

,items,ratings,include,exclude
u,,,,
0,"[1118, 27680, 32541, 137353, 16428, 28815, 658...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 4.0, ...","[6270, 840, 2499, 5010, 6906, 5006, 3203, 7655...","[63, 1168, 335, 7557, 6696, 3217, 4987, 1124, ..."
1,"[122140, 77036, 156817, 76957, 68818, 155600, ...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[6270, 840, 2499, 5010, 6906, 5006, 3203, 7655...","[800, 4253, 7449, 7557, 4623, 7470, 1124, 7946..."
2,"[168054, 87218, 35731, 1, 20475, 9039, 124834,...","[3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 5.0, ...","[6270, 840, 2499, 5010, 6906, 5006, 3203, 7655...","[2499, 5006, 7655, 4717, 5319, 800, 332, 63, 6..."
3,"[163193, 156352, 102888, 19914, 169438, 55772,...","[5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 5.0, 5.0, ...","[6906, 2683, 5319, 800, 6335, 335, 4253, 7557,...",[]
4,"[72857, 38652, 160427, 55772, 119999, 141777, ...","[5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 4.0, 5.0, ...","[5319, 800, 6335, 6276, 1168, 7557, 6654, 590,...",[5648]


In [134]:
def random_select(x): # pick from 0 to x
    result = []
    while len(result)<ingr_count:
        p = int(random.uniform(0, x))
        if p not in result:
            result.append(p)
    return result

In [135]:
for index, row in user_data.iterrows():
    # select 10 for include
    if len(row['include']) > ingr_count:
        pick = random_select(len(row['include'])-1)
        result = []
        for p in pick:
            result.append(row['include'][p])
        row['include'] = result

    # select 10 for exclude
    if len(row['exclude']) > ingr_count:
        pick = random_select(len(row['exclude'])-1)
        result = []
        for p in range (0, len(pick)):
            result.append(row['exclude'][p])
        row['exclude'] = result            

In [136]:
user_data.head()

,items,ratings,include,exclude
u,,,,
0,"[1118, 27680, 32541, 137353, 16428, 28815, 658...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 4.0, ...","[1833, 1257, 335, 5695, 6335]","[63, 1168, 335, 7557, 6696]"
1,"[122140, 77036, 156817, 76957, 68818, 155600, ...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[590, 2832, 6324, 1910, 298]","[800, 4253, 7449, 7557, 4623]"
2,"[168054, 87218, 35731, 1, 20475, 9039, 124834,...","[3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 5.0, ...","[2683, 3497, 7470, 2131, 1329]","[2499, 5006, 7655, 4717, 5319]"
3,"[163193, 156352, 102888, 19914, 169438, 55772,...","[5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 5.0, 5.0, ...","[7367, 1257, 4096, 3440, 4623]",[]
4,"[72857, 38652, 160427, 55772, 119999, 141777, ...","[5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 4.0, 5.0, ...","[5319, 3440, 5825, 800, 5298]",[5648]


In [138]:
ingr_const = user_data.loc[:, ['include', 'exclude']]
ingr_const.head()

,include,exclude
u,,
0,"[1833, 1257, 335, 5695, 6335]","[63, 1168, 335, 7557, 6696]"
1,"[590, 2832, 6324, 1910, 298]","[800, 4253, 7449, 7557, 4623]"
2,"[2683, 3497, 7470, 2131, 1329]","[2499, 5006, 7655, 4717, 5319]"
3,"[7367, 1257, 4096, 3440, 4623]",[]
4,"[5319, 3440, 5825, 800, 5298]",[5648]


In [141]:
ingr_const.to_csv('../data/ingr_const.csv')

## Nutrition related Constraints

In [14]:
data = {
        'calories': [x * mult for x in  calories],
        'total fat': [100 * mult] * nutr_count,
        'sugar': [100 * mult] * nutr_count,
        'sodium': [100 * mult] * nutr_count,
        'protein': [100 * mult] * nutr_count,
        'saturated fat': [100 * mult] * nutr_count,
        'carbohydrates' : [100 * mult] * nutr_count    
       }

In [17]:
nutr_const = pd.DataFrame(data)
nutr_const.head()

,calories,total fat,sugar,sodium,protein,saturated fat,carbohydrates
0,1333.333333,66.666667,66.666667,66.666667,66.666667,66.666667,66.666667
1,1600.000000,66.666667,66.666667,66.666667,66.666667,66.666667,66.666667
2,1866.666667,66.666667,66.666667,66.666667,66.666667,66.666667,66.666667


In [19]:
nutr_const.to_csv('../data/nutr_const.csv', index = False)

In [27]:
recipe_data = pd.read_csv('../data/recipe_data.csv')

In [29]:
recipe_data.shape

(178265, 5)

In [37]:
appear = [0] * len(recipe_data)

In [39]:
for index, row in user_data.iterrows():
    for x in range (0, len(row['items'])):
        item_id = int(row['items'][x])
        appear[item_id] = appear[item_id] + 1

In [61]:
food_count_data = pd.DataFrame(appear)
food_count_data.sort_values(by = 0, ascending = False, inplace = True)
food_count_data.head()

,0
99787,1091
134610,1075
135961,897
117899,894
147374,787


In [64]:
food_list = food_count_data.head(food_count).index.values.tolist()
food_list

[99787,
 134610,
 135961,
 117899,
 147374,
 52334,
 101819,
 37047,
 56425,
 127080,
 147180,
 89113,
 55772,
 15173,
 139822,
 125637,
 50909,
 37359,
 73956,
 28552]

In [66]:
food_list_data = pd.DataFrame(food_list)
food_list_data

,0
0,99787
1,134610
2,135961
3,117899
4,147374
5,52334
6,101819
7,37047
8,56425
9,127080


In [1]:
food_list_data.to_csv('../data/food_const.csv', index = False)

NameError: name 'food_list_data' is not defined